In [ ]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from pathlib import Path

# List your input files here (these may be MP4/M4A containers with .wav names)
files = [
    't11_a1_2025-09-24_16_51.wav',
    't11_e1_2025-09-24_16_51.wav',
    't11_s1_2025-09-24_16_51.wav',
]

# Keep cells short: actual conversion happens in the next cell
print('Files to process:', files)


In [ ]:
from pathlib import Path
import subprocess
import shutil
import soundfile as sf
import numpy as np
import warnings
import os

# Convert files to guaranteed PCM WAVs and validate the output
out_wavs = []

if shutil.which('ffmpeg') is None:
    warnings.warn('ffmpeg not found on PATH. Please install ffmpeg and add it to PATH for reliable conversion.')
else:
    print('ffmpeg found at', shutil.which('ffmpeg'))

for fn in files:
    p = Path(fn)
    if not p.exists():
        p = Path('.') / fn
        if not p.exists():
            warnings.warn(f'File not found: {fn}')
            continue
    out = p.with_suffix('.converted.wav')

    # Use ffmpeg to generate a true PCM 16-bit WAV (mono, 44.1 kHz)
    cmd = [
        'ffmpeg', '-y', '-i', str(p),
        '-vn', '-ac', '1', '-ar', '44100',
        '-acodec', 'pcm_s16le',
        str(out)
    ]
    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if proc.returncode != 0:
        warnings.warn(f'ffmpeg failed for {p.name}: {proc.stderr.splitlines()[-1] if proc.stderr else proc.returncode}')
        continue

    # Quick header check to ensure it's a WAV
    try:
        with open(out, 'rb') as fh:
            hdr = fh.read(12)
        if not (hdr.startswith(b'RIFF') and b'WAVE' in hdr):
            warnings.warn(f'Converted file {out.name} does not look like a WAV (bad header).')
            continue
    except Exception as e:
        warnings.warn(f'Could not read converted file header {out}: {e}')
        continue

    # Read using soundfile to validate format and load data
    try:
        info = sf.info(str(out))
        data, sr = sf.read(str(out), dtype='float32')
        print(f'Converted {p.name} -> {out.name} sr={info.samplerate} channels={info.channels} subtype={info.subtype}')
    except Exception as e:
        warnings.warn(f'Failed to read converted WAV {out}: {e}')
        continue

    if data.ndim > 1:
        data = data.mean(axis=1)

    out_wavs.append({'orig': str(p), 'wav': str(out), 'arr': data, 'sr': int(sr)})

print(f'Conversion finished. {len(out_wavs)} files converted and validated.')


SyntaxError: invalid decimal literal (2102613940.py, line 1)

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

if not out_wavs:
    print('No converted WAVs available to plot.')
else:
    n = len(out_wavs)
    plt.figure(figsize=(10, 2.6 * n))
    for i, info in enumerate(out_wavs, start=1):
        arr = info['arr']
        sr = info['sr']
        t = np.arange(len(arr)) / sr
        ax = plt.subplot(n, 1, i)
        ax.plot(t, arr, linewidth=0.6)
        ax.set_xlim(0, t[-1])
        ax.set_ylabel('Amplitude')
        ax.set_title(f"{Path(info['orig']).name} -> {Path(info['wav']).name} sr={sr}Hz {t[-1]:.2f}s")
        if i < n:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

# Export arrays into notebook globals for later use
for info in out_wavs:
    var = Path(info['wav']).stem.replace('-', '_')
    globals()[var] = info['arr']
    globals()[f"{var}_sr"] = info['sr']

print('Done. Converted WAVs are available and plotted. Variables created for each .converted.wav stem (and _sr).')


FileNotFoundError: [WinError 2] The system cannot find the file specified